In [5]:
%load_ext Cython

In [6]:
import numpy as np
import pandas as pd
y = np.random.randint(2, size=(1000000, 1))
x = np.random.randint(10, size=(1000000, 1))
result = np.zeros(x.shape[0])
#print(result)

print(y.shape)
#print(y)
print(x.shape)
#print(x)
data = pd.DataFrame(np.concatenate([y, x], axis=1), columns=['y', 'x'])
data_numpy = data.to_numpy() 
#print(data_numpy)
#print(data)
#print(data_numpy[:,1])


(1000000, 1)
(1000000, 1)


In [7]:
def target_mean_v1(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    for i in range(data.shape[0]):
        groupby_result = data[data.index != i].groupby([x_name], as_index=False).agg(['mean', 'count'])
        result[i] = (groupby_result.loc[groupby_result.index == data.loc[i, x_name], (y_name, 'mean')])
    return result



def target_mean_v2(data, y_name, x_name):
    result = np.zeros(data.shape[0])
    value_dict = dict()
    count_dict = dict()
    for i in range(data.shape[0]):
        if data.loc[i, x_name] not in value_dict.keys():
            value_dict[data.loc[i, x_name]] = data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] = 1
        else:
            value_dict[data.loc[i, x_name]] += data.loc[i, y_name]
            count_dict[data.loc[i, x_name]] += 1
    for i in range(data.shape[0]):
        result[i] = (value_dict[data.loc[i, x_name]] - data.loc[i, y_name]) / (count_dict[data.loc[i, x_name]] - 1)
    return result

In [8]:

%%cython --cplus 

cimport cython
cimport numpy as cnp
from libcpp.vector cimport vector
from libcpp.map cimport map
cpdef target_mean_v3(cnp.ndarray[long] y_values, cnp.ndarray[long] x_values, cnp.ndarray[double] result):
    cdef int sampleSize = x_values.size
    cdef map[long, double] value_dict
    cdef map[long, int] count_dict
    for i in range(sampleSize):
        if value_dict.find(x_values[i]) ==  value_dict.end():
            value_dict[x_values[i]] = y_values[i]
            count_dict[x_values[i]] = 1
        else:
            value_dict[x_values[i]] += y_values[i]
            count_dict[x_values[i]] += 1
    #print(value_dict)
    #print(count_dict)
    for i in range(sampleSize):
      if count_dict[x_values[i]] - 1 != 0 :
        tmp = (value_dict[x_values[i]] - y_values[i]) / (count_dict[x_values[i]] - 1)
        result[i] = tmp
      else:
        result[i] = 0.0
    return result

In [12]:
%%cython --cplus 

cimport cython
cimport numpy as cnp
from libcpp.vector cimport vector
from libcpp.unordered_map cimport unordered_map
cpdef target_mean_v4(cnp.ndarray[long] y_values, cnp.ndarray[long] x_values, cnp.ndarray[double] result):
    cdef int sampleSize = x_values.size
    cdef unordered_map[long, double] value_dict
    cdef unordered_map[long, int] count_dict
    for i in range(sampleSize):
        if value_dict.find(x_values[i]) ==  value_dict.end():
            value_dict[x_values[i]] = y_values[i]
            count_dict[x_values[i]] = 1
        else:
            value_dict[x_values[i]] += y_values[i]
            count_dict[x_values[i]] += 1
    #print(value_dict)
    #print(count_dict)
    for i in range(sampleSize):
      if count_dict[x_values[i]] - 1 != 0 :
        tmp = (value_dict[x_values[i]] - y_values[i]) / (count_dict[x_values[i]] - 1)
        result[i] = tmp
      else:
        result[i] = 0.0
    return result

In [10]:
%%cython -a

cimport cython
cimport numpy as cnp
cpdef target_mean_v5(cnp.ndarray[long] y_values, cnp.ndarray[long] x_values, cnp.ndarray[double] result):
    cdef int sampleSize = x_values.size
    cdef int value_dict[10]
    cdef int count_dict[10]
    value_dict[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    count_dict[:] = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    for i in range(sampleSize):
        value_dict[x_values[i]] += y_values[i]
        count_dict[x_values[i]] += 1
    #print(value_dict)
    #print(count_dict)
    for i in range(sampleSize):
      if count_dict[x_values[i]] - 1 != 0 :
        tmp = (value_dict[x_values[i]] - y_values[i]) / (count_dict[x_values[i]] - 1)
        result[i] = tmp
      else:
        result[i] = 0.0
    return result

In [13]:
import time
start = time.time()
#long running
#do something other
result_3= target_mean_v3(data_numpy[:,0], data_numpy[:,1], result)
end = time.time()
print(str((end-start)*1000) + "ms")
print(result_3)

83.25767517089844ms
[0.49754355 0.499421   0.49754355 ... 0.49754355 0.49941101 0.4992038 ]


In [14]:
import time
start = time.time()
#long running
#do something other
result_4 = target_mean_v4(data_numpy[:,0], data_numpy[:,1], result)
end = time.time()
print(str((end-start)*1000) + "ms")
print(result_4)
#print(result_3)

93.42837333679199ms
[0.49754355 0.499421   0.49754355 ... 0.49754355 0.49941101 0.4992038 ]


In [15]:
%%timeit
result_3 = target_mean_v3(data_numpy[:,0], data_numpy[:,1], result)

10 loops, best of 3: 83.7 ms per loop


In [16]:
%%timeit
result_4 = target_mean_v4(data_numpy[:,0], data_numpy[:,1], result)

10 loops, best of 3: 93.3 ms per loop


In [17]:
%%timeit
result_5 = target_mean_v5(data_numpy[:,0], data_numpy[:,1], result)

100 loops, best of 3: 10.8 ms per loop


In [18]:
%%timeit
result_2 = target_mean_v2(data, 'y', 'x')

KeyboardInterrupt: ignored

In [13]:
%%timeit
result_1 = target_mean_v1(data, 'y', 'x')

KeyboardInterrupt: ignored

In [ ]:
import time
start = time.time()
#long running
#do something other
result_2 = target_mean_v2(data, 'y', 'x')
end = time.time()
print(str((end-start)*1000) + "ms")
print(result_2)
#print(result_3)

2.4411678314208984ms
[0.16666667 0.55555556 0.66666667 0.75       0.16666667 0.75
 0.66666667 0.66666667 0.44444444 1.         0.14285714 0.16666667
 0.14285714 0.         0.14285714 0.66666667 0.55555556 0.44444444
 0.55555556 0.83333333 0.16666667 0.14285714 0.5        1.
 0.16666667 0.14285714 0.66666667 0.66666667 0.44444444 0.
 0.83333333 0.5        1.         0.66666667 0.         0.75
 0.14285714 0.55555556 0.44444444 0.44444444 1.         0.
 0.         0.66666667 0.         0.16666667 0.55555556 0.75
 0.14285714 0.        ]


In [ ]:
import time
start = time.time()
#long running
#do something other
result_1 = target_mean_v1(data, 'y', 'x')
end = time.time()
print(str((end-start)*1000) + "ms")
print(result_1)
#print(result_3)

414437.04867362976ms
[0.4918066  0.50326931 0.50040388 ... 0.50210801 0.50809191 0.49681036]


In [21]:
    result_5 = target_mean_v5(data_numpy[:,0], data_numpy[:,1], result)
    diff = np.linalg.norm(result_5 - result_4)
    print(diff)

0.0
